<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/predictions-final/Rookie_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd

In [0]:
# Load the data
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/players_full.csv')
kickers2019_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie2019_kickers.csv')
offense2019_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_rookie2019_offense.csv')
rookies_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies.csv')
bye_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/bye.csv')
injury_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/injury.csv')
actuals2019_df = pd.concat([kickers2019_df, offense2019_df], ignore_index=True)

In [3]:
# The dataframe of all 2019 players
player_df.head()

,player,first,last,name,position1,start,cteam,height,weight,dob,forty,bench,vertical,broad,shuttle,cone,arm,hand,dpos,col,dv,jnum,dcp
0,SF,San Francisco,49ers,San Francisco 49ers,DF,2000,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHI,Chicago,Bears,Chicago Bears,DF,2000,CHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,2000,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUF,Buffalo,Bills,Buffalo Bills,DF,2000,BUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEN,Denver,Broncos,Denver Broncos,DF,2000,DEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# The dataframe for all 2019 rookies
rookies_df.head()

,player,first,last,name,position1,start
0,MG-0306,Matt,Gay,Matt Gay,K,2019
1,CM-2137,Chase,McLaughlin,Chase McLaughlin,K,2019
2,AS-0575,Austin,Seibert,Austin Seibert,K,2019
3,JS-3825,Joey,Slye,Joey Slye,K,2019
4,DB-2750,David,Blough,David Blough,QB,2019


In [5]:
# The dataframe of 2019 actual points for each 2019 rookie
actuals2019_df.head()

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,MG-0306,Matt,Gay,Matt Gay,K,5.0,8.0,15.0,15.0,7.0,6.0,10.0,12.0,13.0,7.0,5.0,8.0,9.0,9.0,10.00,1.00
1,CM-2137,Chase,McLaughlin,Chase McLaughlin,K,8.0,5.0,10.0,5.0,11.0,6.0,14.0,10.0,1.0,7.0,10.0,0.0,0.0,0.0,0.00,0.00
2,AS-0575,Austin,Seibert,Austin Seibert,K,1.0,13.0,7.0,11.0,3.0,2.0,8.0,13.0,8.0,3.0,10.0,6.0,11.0,7.0,3.00,6.00
3,JS-3825,Joey,Slye,Joey Slye,K,11.0,16.0,8.0,13.0,2.0,14.0,4.0,6.0,5.0,3.0,10.0,2.0,10.0,8.0,8.00,4.00
4,DB-2750,David,Blough,David Blough,QB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.3,8.2,8.3,8.68,11.88


In [6]:
# The dataframe of bye weeks for 2019 teams
bye_df.head()

,player,first,last,name,position1,bye-week
0,SF,San Francisco,49ers,San Francisco 49ers,DF,4
1,CHI,Chicago,Bears,Chicago Bears,DF,6
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,9
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6
4,DEN,Denver,Broncos,Denver Broncos,DF,10


In [7]:
# The dataframe of injuries for 2019 players
injury_df.head()

,uid,gid,year,week,player,first,last,name,position1,team,details,pstat,gstat
0,29844,5194,2019,10,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
1,30530,5213,2019,11,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
2,31097,5228,2019,12,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
3,31629,5242,2019,13,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR
4,32187,5256,2019,14,AA-0581,Antony,Auclair,Antony Auclair,TE,TB,Toe,NaN,IR


In [0]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each team
def add_row(df, p, f, l, n, pos, wp, wa, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position': pos,
                    wp: pred,
                    wa: act
                    }, ignore_index=True)
    
    return df

In [0]:
# The main code for iterating through the player(rookie) list, calculating the points and adding the rows
# to the final_df dataframe.
def main(w):
    week_pname = 'week' + str(w) + '-pred'
    week_aname = 'week' + str(w) + '-act'

    column_names = ['player',
                    'first',
                    'last',
                    'name',
                    'position', 
                    week_pname,
                    week_aname
                     ]

    player_list = rookies_df['player'].tolist()
    row = 0
    col = 4 + w

    week_df = pd.DataFrame(columns = column_names)

    for player in player_list:

        first = rookies_df['first'].loc[(rookies_df['player']==player)].iloc[0]
        last = rookies_df['last'].loc[(rookies_df['player']==player)].iloc[0]
        name = rookies_df['name'].loc[(rookies_df['player']==player)].iloc[0]
        position1 = rookies_df['position1'].loc[(rookies_df['player']==player)].iloc[0]
        team = player_df['cteam'].loc[(player_df['player']==player)].iloc[0]

        injury_weeks = injury_df['week'].loc[(injury_df['player']==player)]
        if (len(injury_weeks)==0):
            injury_week = 18
        else:
            injury_week = min(injury_weeks)

        bye_week = bye_df['bye-week'].loc[(bye_df['player']==team)].iloc[0]

        if w > bye_week:
            new_col = col - 1
        else:
            new_col = col

        act_points = actuals2019_df.loc[(actuals2019_df['player']==player)].iloc[0, new_col:21].sum()
        row = row + 1
    
        # Forecast
        if w > bye_week:
            n_periods = 17 - w + 1
        else:
            n_periods = 17 - w

        end_col = 5 + w - 1
        act_avg = actuals2019_df.loc[(actuals2019_df['player']==player)].iloc[0, 5:end_col].mean()
        if (injury_week <= w):
            pred_points = 0
        else:
            pred_points = n_periods * act_avg

        week_df = add_row(week_df, player, first, last, name, position1, week_pname, week_aname, pred_points, act_points)
    return week_df

In [0]:
week = 14
final_df = main(week)

In [11]:
final_df.head(50)

,player,first,last,name,position,week14-pred,week14-act
0,MG-0306,Matt,Gay,Matt Gay,K,36.923077,29.00
1,CM-2137,Chase,McLaughlin,Chase McLaughlin,K,26.769231,0.00
2,AS-0575,Austin,Seibert,Austin Seibert,K,29.538462,27.00
3,JS-3825,Joey,Slye,Joey Slye,K,32.000000,30.00
4,DB-2750,David,Blough,David Blough,QB,8.153846,37.06
5,DH-1912,Dwayne,Haskins,Dwayne Haskins,QB,13.950769,43.06
6,DH-3450,Devlin,Hodges,Devlin Hodges,QB,14.393846,19.02
7,DJ-2275,Daniel,Jones,Daniel Jones,QB,0.000000,49.92
8,DL-1787,Drew,Lock,Drew Lock,QB,0.000000,55.14
9,GM-1350,Gardner,Minshew,Gardner Minshew,QB,53.280000,72.66


In [0]:
week_pred = 'week' + str(week) + '-pred'
week_act = 'week' + str(week) + '-act'
week_diff = 'week' + str(week) + '-diff'
week_pct = 'week' + str(week) + '-pct'

In [0]:
# The final_df dataframe
final_df[week_diff] = final_df[week_pred] - final_df[week_act]
final_df[week_pct] = final_df[week_diff] / final_df[week_pred]
final_df[week_pred] = final_df[week_pred].astype(int)

In [14]:
# Calculate the metrics
pred_median_error = final_df[week_pct].median()

print('Median Error - %.4f%%' % (pred_median_error * 100))

Median Error - -83.9433%


In [15]:
final_df.head(50)

,player,first,last,name,position,week14-pred,week14-act,week14-diff,week14-pct
0,MG-0306,Matt,Gay,Matt Gay,K,36,29.00,7.923077,0.214583
1,CM-2137,Chase,McLaughlin,Chase McLaughlin,K,26,0.00,26.769231,1.000000
2,AS-0575,Austin,Seibert,Austin Seibert,K,29,27.00,2.538462,0.085938
3,JS-3825,Joey,Slye,Joey Slye,K,32,30.00,2.000000,0.062500
4,DB-2750,David,Blough,David Blough,QB,8,37.06,-28.906154,-3.545094
5,DH-1912,Dwayne,Haskins,Dwayne Haskins,QB,13,43.06,-29.109231,-2.086568
6,DH-3450,Devlin,Hodges,Devlin Hodges,QB,14,19.02,-4.626154,-0.321398
7,DJ-2275,Daniel,Jones,Daniel Jones,QB,0,49.92,-49.920000,-inf
8,DL-1787,Drew,Lock,Drew Lock,QB,0,55.14,-55.140000,-inf
9,GM-1350,Gardner,Minshew,Gardner Minshew,QB,53,72.66,-19.380000,-0.363739
